In [1]:
import numpy as np
import time

In [2]:
# This function is to find the adjacent vertices of vertex in graph R. E is adjacency matrix
def adjV(p, R, E):
    tp = []
    for e in R:
        if E[p - 1][e - 1] == 1: # if vertex e has edge with p
            tp.append(e)
    return tp

In [3]:
#This function is to count the degree of a vertex
def degV(p, R, E):
    d = 0
    for e in R:
        if E[p - 1][e - 1] == 1:
            d = d + 1
    return d
    

In [4]:
#Find the vertices with the minimum degree in graph R
def mindeg(R, E):
    Rmin = []
    mindeg = np.inf
    for v in R:         #find min degree
        tmp = degV(v, R, E)
        if tmp < mindeg:
            mindeg = tmp
    for v in R:         # find vertice whose degree is min
        if degV(v, R, E) ==mindeg:
            Rmin.append(v)
    return Rmin


In [5]:
#Calculate the exdegree of certain vertex
def exdegV(p, R, E):
    mindeg = np.inf
    Tp = adjV(p, R, E)# find adjacent vertices
    sumdeg = 0
    for v in Tp:    #use degV function to help calculate the exdegree
        sumdeg = sumdeg + degV(v, R, E)
    return sumdeg    

In [6]:
#Find the vertex whoes degree is minimum
def minexdeg(R, E):
    exdegs = []
    for v in R:
        exdegs.append(exdegV(v, R, E))
    i = exdegs.index(min(exdegs))
    return R[i]

In [7]:
# To store the max clique during the recursion of expand function
class Qm:
    def __init__(self, q):
        self.q = q       
    def visit(self):
        return self.q
    def length(self):
        return len(self.q)
    def modify(self, new):
        self.q = new
       

In [8]:
def expand(R, No, No_ini, Q, q, E): #R is graph, No is number of each vertex, No_ini is the initial number, Q is clique,q is max clique
    while len(R) != 0:#while not empty
        p = R[-1]  #rightmost vertex
        if len(Q) + No[p] > q.length() :
            Q.append(p)

            Rp = adjV(p, R, E)
            if len(Rp) != 0:

                NUMSORT(Rp, No_ini, E)
                
                expand(Rp, No_ini, No_ini, Q, q, E)
            else:
                if(len(Q) > q.length()):
                    tmp = [h for h in Q]
                    q.modify(tmp)
            Q.pop(-1)
        else:
            return
        R.pop(-1)
    


In [9]:
def NUMSORT(R, No, E):
    #NUMBER
    maxno = 0
    r = [v for v in R]
    C = []
    C.append(False)
    C.append([])
    while(len(R)!=0):
        p = R[0]
        Tp = adjV(p, r, E)
        k = 1
        while len([v for v in C[k] if v in Tp]) != 0:
            k = k + 1
        if k > maxno:
            maxno = k
            C.append([])
        No[p] = k
        C[k].append(p)
        R.pop(0)
        
    #SORT
    for k in range(1, maxno + 1):
        for j in range(len(C[k])):
            R.append(C[k][j])


In [10]:
def MCR(Vertex, E):
    #Global
    Q = []
    Qmax = []
    
    #Sort
    i = len(Vertex)
    R = [v for v in Vertex]
    V = []
    while(len(V)!= i):
        V.append(0)
    
    
    deg = [degV(v, R, E) for v in R]#degree list of each vertex
    Rmin = mindeg(R, E)
        
    while(len(Rmin) != len(R)):
        if(len(Rmin) >= 2):
            p = minexdeg(Rmin, E)
            
        else:
            p = Rmin[0]
        V[i - 1] = p
        R.remove(p)
        i = i - 1
        for j in range(len(R)):
            if E[p - 1][R[j] - 1] == 1:
                deg[R[j] - 1] = deg[R[j] - 1] - 1
        
        Rmin = mindeg(R, E)
    
    #Regular subgraph
    No = []
    for i in range(len(E) + 1):
        No.append(0)
    NUMSORT(Rmin, No, E)
    for i in range(len(Rmin)):
        V[i] = Rmin[i]
    
    #Number
    tmp = 0
    for r in Rmin:
        if No[r] > tmp:
            tmp = No[r]
    m = tmp
    AG = max(deg)
    mmax = len(Rmin) + AG - m
    m = m + 1
    i = len(Rmin) + 1
    while(i <= mmax):
        if(i > len(V)):
            break
        No[V[i - 1]] = m
        m = m + 1
        i = i + 1
    if(i < len(V)):
        for i in range(mmax, len(V)):
            No[V[i]] = AG + 1
#Start
    judge = True
    for q in Rmin:
            if degV(q,R, E) != len(Rmin) - 1:
                judge = False
    if(judge == True):
            Qmax = Rmin
    q = Qm(Qmax)#instantialize class Qm
    No1 = []
    for i in range(len(E) + 1):
        No1.append(0)
    expand(V, No, No1, Q, q, E)
    print('Cardinality:', q.length())
    print("Maximum clique:",q.visit())


In [11]:
# Reading .txt file, generate adjacent matrix
adj = np.loadtxt("p_hat300_3.txt", dtype = int)
size = adj.max()
a = np.zeros([size,size])
for e in adj:
    a[e[0] - 1][e[1] - 1] = 1
    a[e[1] - 1][e[0] - 1] = 1
    
v = [i for i in range(1, size + 1)]
t1 = time.perf_counter()#begin time
MCR(v, a)  #main
t2 = time.perf_counter()#end time
print("Time for algorithm to solve(in CPU seconds):",t2 - t1)

Cardinality: 36
Maximum clique: [18, 176, 138, 293, 299, 20, 199, 166, 4, 33, 98, 219, 239, 298, 48, 290, 76, 139, 21, 255, 252, 174, 245, 226, 221, 205, 273, 190, 247, 19, 235, 89, 149, 56, 49, 40]
Time for algorithm to solve(in CPU seconds): 68.4305089999998
